# LMDZ GCM
[LMDZ](http://lmdz.lmd.jussieu.fr/le-projet-lmdz/formation/2017)
[mars?](http://www-mars.lmd.jussieu.fr/)
## Download, compile

In [1]:
%%bash
run=svn && which $run || sudo apt-get install subversion || true
which $run || echo "Please install svn..."
INST="svn co http://svn.lmd.jussieu.fr/LMDZ/BOL/script_install && cd script_install && chmod +x install*.sh && ./install*.sh"
echo $INST

/usr/bin/svn
/usr/bin/svn
svn co http://svn.lmd.jussieu.fr/LMDZ/BOL/script_install && cd script_install && chmod +x install*.sh && ./install*.sh


In [3]:
######################
from __future__ import print_function
import os, pwd, sys; 
lib = "/usr/local/apps/ecflow/current/lib/python2.7/site-packages/ecflow"
lib = "/usr/local/apps/ecflow/current/lib/python3.5/site-packages/ecflow"
lib = "/usr/local/lib/python3.5/site-packages/ecflow"
sys.path.append(lib)
import ecf; from ecf import (Client, Defs, Suite, Family, Task, Defstatus, Edit, Trigger)
try: x = Edit(test="value")  # Edit is present in recent ecf.py module
except:
    class Edit(Variables): pass
home = os.getenv("HOME") + "/ecflow_server"
user = os.getenv("USER")

# SUITE
node = Suite("lmdz").add(
    Defstatus("suspended"),
    Edit(ECF_HOME=home, ECF_INCLUDE=home + "/include", ECF_FILES=home + "/files",
         ECF_EXTN=".ecg",  # current convention for generated task template extension
         ECF_JOB_CMD="%ECF_JOB% > %ECF_JOBOUT% 2>&1",  # localhost run
         ECF_URL_CMD="firefox %URL%",
         URL="http://lmdz.lmd.jussieu.fr/", ),
    Family("make").add(
        Family("get").add(Task("cmd").add(
            Edit(CMD="svn co http://svn.lmd.jussieu.fr/LMDZ/BOL/script_install",
                 ARGS="")), ),
        Family("compile").add(
            Trigger(["get"]),
            Task("cmd").add(
                Edit(CMD="cd script_install && chmod +x install*sh && ./install_lmdz.sh")), ), ),
    Family("main").add(Task("cmd").add(Edit(CMD="echo", ARGS="YOUR PART"))))

# TASK TEMPLATE
with open(home + "/files/cmd.ecg", 'w') as task_template: print("""#!/bin/bash
%include <head.h>
 %CMD:echo% %ARGS:%
%include <tail.h>""", file=task_template)

# DEFS
defs = Defs()
defs.add_suite(node)
path = '/' + node.name()

# CLIENT
client = Client("localhost@%s" % os.getenv("ECF_PORT", 2500))  # PYTHON CLIENT
client.replace(path, defs)  # load/replace the top node (suite)
client.begin_suite(node.name())  # BEGIN suite: UNKNOWN -> QUEUED
client.resume(path) # RESUME suite: SUSPENDED -> create job and submit